# 

# Azione
- Pag. partenza (opzionale)
- Target (XPATH)
- Operazione (CLICK, WRITE, READ)
- NextOP → LinkTo *Action (opzionale)

In [1]:
import time
import types
from copy import deepcopy

import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import ScraperEngine.scraper_engine as _SE
import ScraperEngine.scrape_operations as _SOp
from ScraperEngine.scrape_operations import OutputOperations as _OOp
from ScraperEngine.scrape_actions import CommonActions, Action, Procedure, Iterated

In [2]:
urls = {
    'ig': {
        'base':                         'https://www.instagram.com/',
        'explore_tags':                 'https://www.instagram.com/explore/tags/'
    },
    'x': {
        'base':                         'https://twitter.com/'
    }
}
xpaths = {
    'main_div': '/html/body/div[2]',
    'outside_signin_popup':             '/html/body/div[6]/div[1]/div/div[2]/div',
    'login_procedure': {
        'allow_cookies_button':         '/html/body/child::div/div[1]/div/div[2]/div/div/div/div/div[2]/div/button[1]', 
        'username_input':               '//*[@id="loginForm"]/div/div[1]/div/label/input',
        'password_input':               '//*[@id="loginForm"]/div/div[2]/div/label/input',
        'submit_button':                '//*[@id="loginForm"]/div/div[3]/button',
        'save_info_button':             '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div/div/div/div',
        'notification_button':          '/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/button[2]',
    },
    'search': {
        'search_icon':                  '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[2]/div[2]/span/div/a',
        'search_input':                 '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div[1]/div/input',
        'account_results':              '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div[2]/div/child::div/a/div/div/div/div[2]/div/div/div/span'
    },
    'explore': {
        'tags': {
            'post_ids':                 '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/article/div/div/div/child::div/child::div/a'
        }                       
    },
    'profile': {
        'post_ids': [
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[2]/section/main/div/div[2]/article/div/div/child::div/child::div/a', 
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/div[2]/article/div[1]/div/child::div/child::div/a',
        ],
        'more_posts_button':            '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[2]/section/main/div/div[2]/div[1]/div/button'
    },
    'post': {
        'author': [
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/span/div/div/span[1]/div/a/div/div/span',
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/span/div/div/div/div/a/div/div/span',                                               
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div[1]/article/div/div[2]/div/div[2]/div[1]/ul/div/li/div/div/div[2]/h2/div[1]/a', 
        ],
        'media_url': [                  1,   
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[1]/div/div/div/div/div/div/div[1]/img',
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[1]/div/div/div/div/div/div[1]/div[2]/div/div/div/ul/li[2]/div/div/div/div/div[1]/img',
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div/article/div/div[1]/div/div/div/div[1]/img',
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div/article/div/div[1]/div/div/div/div/div/div/div/div/div/img',
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/video',
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div/article/div/div[1]/div/div/div/div/div/div/div/video'
        ],        
        'description': [
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div[1]/article/div/div[2]/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/h1', 
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/span/div/span',   
        ],
        'datetime': [
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div[1]/article/div/div[2]/div/div[2]/div[2]/div/div/a/span/time', 
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[3]/div[2]/div/a/span/time',
        ],
        'likes': {
                                        '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/section/main/div[1]/div[1]/article/div/div[2]/div/div[2]/section[2]/div/div/span/a/span/span'
        }
    }
}

x_xpath = {
    'login': {
        'allow_cookies':                '/html/body/div/div/div/div[1]/div/div/div/div/div/div[2]/div[1]',
        'google_signup':                '/html/body/div/div/div/div[2]/main/div/div/div[1]/div[1]/div/div[3]/div[1]'
    }
}

In [4]:
""" Instagram actions """

open_ig = CommonActions.OpenPage(urls['ig']['base'])
allow_cookies = Action('allow_cookies', urls['ig']['base'], xpaths['login_procedure']['allow_cookies_button'], _SOp.ClickOp())

login = Procedure('login', [
    Action('insert_username', urls['ig']['base'], xpaths['login_procedure']['username_input'], _SOp.WriteOp(lambda: input('ig user')),
    Action('insert_password', urls['ig']['base'], xpaths['login_procedure']['password_input'], _SOp.WriteOp(lambda: input('ig pwd'))),
    Action('submit', urls['ig']['base'], xpaths['login_procedure']['submit_button'], _SOp.ClickOp()),
    Action('no_save_info', urls['ig']['base'], xpaths['login_procedure']['save_info_button'], _SOp.ClickOp()),
    Action('no_notification', urls['ig']['base'], xpaths['login_procedure']['notification_button'], _SOp.ClickOp()),
])

ig_access = Procedure('ig_access', [
    open_ig, 
    allow_cookies,
    login
])

search_for = Procedure('search_for', [
    Action('write_search', urls['ig']['base'], xpaths['search']['search_input'], _SOp.WriteOp('travel', True)),
    Action('click_search', urls['ig']['base'], xpaths['search']['search_icon'], _SOp.ClickOp()),
    Action('get_account_results', urls['ig']['base'], xpaths['search']['account_results'], _SOp.ReadOp(is_multiple=True))
])

scan_post_ids = lambda posts_xpath: Procedure('scan_post_ids', [
    Action('get_post_links', None, posts_xpath, _SOp.ReadOp('href', is_multiple=True)),
    CommonActions.Scroll('scroll_posts'),
    CommonActions.Wait('wait_posts_load', 3)
])  

explore_tag = Procedure('explore_tag', [
    CommonActions.OpenPage(lambda: urls['ig']['explore_tags'] + input('Search tag')),
    Iterated(scan_post_ids(xpaths['explore']['tags']['post_ids']), 2)
])

get_profile_posts = Procedure('profile_posts', [
    CommonActions.OpenPage(lambda: urls['ig']['base'] + input('Search profile')),
    Iterated(scan_post_ids(xpaths['profile']['post_ids']), 1)
], out_op=lambda res: list({pid for scan_iter in res['scan_post_ids'] for pid in scan_iter['get_post_links']}))

get_post_details = lambda pid: Procedure('post_details', [
    CommonActions.OpenPage(pid),
    Action('get_author', None, xpaths['post']['author'], _SOp.ReadOp()),
    Action('get_description', None, xpaths['post']['description'], _SOp.ReadOp()),
    Action('get_datetime', None, xpaths['post']['datetime'], _SOp.ReadOp('datetime')),
])

get_posts_details = Iterated(
    Procedure('post_details', [
        Action('open_page', lambda pid: pid, None, None),
        Action('get_author', None, xpaths['post']['author'], _SOp.ReadOp()),
        Action('get_media_url', None, xpaths['post']['media_url'], _SOp.ReadOp('src')),
        Action('get_description', None, xpaths['post']['description'], _SOp.ReadOp()),
        Action('get_likes', None, xpaths['post']['likes'], _SOp.ReadOp()),
        Action('get_datetime', None, xpaths['post']['datetime'], _SOp.ReadOp('datetime')),
    ]),
    times = lambda: len(get_profile_posts.output),
    iter_args = lambda: [{'pid': pid} for pid in get_profile_posts.output],        
)

In [5]:
scr = _SE.Scraper('igs', Procedure('xxx', [
        ig_access,
        get_profile_posts, 
        get_posts_details
    ], 
    out_op = lambda out: _OOp.make_df(data = [{'pid': pid, **details} for pid, details in zip(out['profile_posts'], out['post_details'])])
))
scr.build()

In [6]:
scr.run_procedure()

run action: open_url
run action: allow_cookies
run action: insert_username
run action: insert_password
run action: submit
run action: no_save_info


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=118.0.5993.88)


In [27]:
scr.procedure.output.head(2)

,pid,open_page,get_author,get_media_url,get_description,get_likes,get_datetime
0,https://www.instagram.com/p/CyirLcjqXd0/,None,matteosalviniofficial,blob:https://www.instagram.com/9f85f017-4e22-400d-a0e0-6d2793208cba,"“Loro in contromano su una moto appena rubata, lui alla guida di una Panda verso l’abitazione della fidanzata. All’incrocio la tragedia: uno schianto violentissimo, l’auto che si ribalta.\n\nAll’interno c’è Lorenzo Brogioni, 43 anni, venditore ambulante di libri. Ha da poco lasciato la casa della madre malata. Era stato da lei per accudirla come faceva frequentemente. Lorenzo rimane schiacciato nella vettura. L’incidente per lui è fatale.\n\nSulla moto che viaggiava a tutta velocità in contromano nel centro della città erano addirittura in tre. Due uomini extracomunitari non riescono a fuggire e sono ora in ospedale, controllati dalle Forze dell’Ordine. La terza persona invece riesce a dileguarsi e al momento è ricercata”, scrive Il Tirreno.\n\nPazzesco.\nLadri, criminali e assassini, da mandare in carcere senza sconti.\nUna preghiera per Lorenzo.",None,2023-10-18T13:27:34.000Z
1,https://www.instagram.com/p/CygWH-HKQwH/,None,matteosalviniofficial,https://scontent.cdninstagram.com/v/t39.30808-6/384198073_871807807846803_9008570298920868301_n.jpg?stp=dst-jpg_e15&cb=d185f762-417afcc3&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xODIweDIwNDguc2RyIn0&_nc_ht=scontent.cdninstagram.com&_nc_cat=1&_nc_ohc=rXRejXTwgIMAX9mSWvA&edm=APs17CUAAAAA&ccb=7-5&ig_cache_key=MzIxNTY2NzQzODY5NDIzOTIzOQ%3D%3D.2-ccb7-5&oh=00_AfDH1jiuhAtC0SQDVLLKuHAWYhMcwAuoPCvO04yrHhVVTg&oe=65331D64&_nc_sid=10d13b,"E io sono stato mandato a PROCESSO dalla sinistra per averli fermati!\nRischio 15 anni di CARCERE per aver difeso e protetto l’Italia.\n\n“I terroristi non arrivano sui barconi"" dicevano…\nQuando ero ministro dell’Interno ho fatto di tutto per stanare le infiltrazioni terroriste e islamiste in Italia e in Europa, bloccando gli sbarchi, fermando il business dell'accoglienza e combattendo criminali e trafficanti.\nRisultato: la stessa sinistra che ha sempre negato i pericoli dell'immigrazione selvaggia mi ha mandato a processo.\nImpedire gli arrivi e rimpatriare chi ci porta la guerra in casa: questo è e sarà l'impegno della Lega e di tutto il governo. Anche se qualcuno non sarà d’accordo…",None,2023-10-17T15:44:13.000Z
